In [1]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import time
from pprint import pprint
import scipy.stats as st
import os.path
from os import path
import json

# Incorporated citipy to determine city based on latitude and longitude
from citipy import citipy

# Output File (CSV)
output_data_players_Titled = "output_data/players_Titled.csv"

country_filepath = "resources/countries.csv"

titled_players_type = ["GM","WGM","IM","WIM","FM","WFM","NM","WNM","CM","WCM"]

# Range of latitudes and longitudes
# lat_range = (-90, 90)
# lng_range = (-180, 180)

In [2]:
# function get usernames by country, pass country code and output file
# returns list of usernames for specific country and saves output file to csv
def get_usernames_country(country, output_file):
    try: 
        # country code # countries list resource https://en.wikipedia.org/wiki/ISO_3166-1_alpha-2
        # country = "RU" # RUSSIA = RU

        query_url = f"https://api.chess.com/pub/country/{country}/players"

        print(f"Fetching player list for country {country}")

        
        # Get player usernames for country
        username_response = requests.get(query_url)
        username_json = username_response.json()        
        # username_json
        
        usernames = []
        for username in username_json["players"]:
            usernames.append(username)

        player_dict = {    
            "username":usernames
        }
        chessplayers_df = pd.DataFrame(player_dict)
        chessplayers_df.to_csv(output_file, index=True,index_label="player_index")          
        
        return chessplayers_df                
    except: 
        print("Unexpected error.")

In [3]:
# pass list of player types
# return player data json for each type of player 
def get_titledplayers_api(playertypes):
    url = "https://api.chess.com/pub/titled/"
    
    i = 0        
    players_data = []
    
    for playertype in playertypes:
        print(f"Index {str(i)} fetching player list for titled player type {playertype}")
    
        # Build query URL
        query_url = f"{url}{playertype}"

        # increment index counter
        i += 1

        # Get weather data    
        playertype_response = requests.get(query_url)
        playertype_json = playertype_response.json()
        players_data.append(playertype_json)        
        # time.sleep()    
        
    return players_data

In [4]:
# pass list of players 
# return player_data json player details
def get_playerdata_api(player_usernames):
    url = "https://api.chess.com/pub/player/"
    
    i = 0        
    player_data = []
    
    for player_username in player_usernames:
        print(f"Index {str(i)} fetching player data for player {player_username}")
    
        # Build query URL
        query_url = f"{url}{player_username}"

        # increment index counter
        i += 1

        # Get player data    
        try:
            player_response = requests.get(query_url)
            player_json = player_response.json()
            player_data.append(player_json)        
            # time.sleep(1)    
        except:
            print("unexpected error")
            
    return player_data

In [5]:
# pass playerdata json
# return dataframe with player details
def get_player_dataframe(playerdata):                
    player_id = []
    player_username = []
    title = []
    followers = []
    country = []
    location = []
    last_online = []
    joined = []
    status = []
    is_streamer = []

    i = 0

    for player in playerdata: 
        try:            
            username = player["username"]
            print(f"Index {i} processing username {username}")
            player_id.append(player["player_id"])
            player_username.append(username)
            if "title" in player.keys():
                title.append(player["title"])
            else:
                title.append("")          
            followers.append(player["followers"])        
            country.append(player["country"].replace("https://api.chess.com/pub/country/","")),
            if "location" in player.keys():
                location.append(player["location"])        
            else:
                location.append("")                
            last_online.append(player["last_online"])
            joined.append(player["joined"])
            status.append(player["status"])
            if "is_streamer" in player.keys():
                is_streamer.append(player["is_streamer"])                       
            else:     
                is_streamer.append("")
        except:
            print("unexpected error")

        i += 1 

    player_dict = {
        "player_id":player_id,
        "username":player_username,
        "title":title,
        "followers":followers,
        "country":country,
        "location":location,
        "last_online":last_online,
        "joined":joined,
        "status":status,
        "is_streamer":is_streamer
    }
    chessplayer_details_df = pd.DataFrame(player_dict)

    # chessplayer_df.to_csv(output_data_players, index=True,index_label="player_index") 
    
    return chessplayer_details_df

In [6]:
# api calls for each country in country_filepath
# generates csv file with list of usernames for each country output_data/players_{country_code}.csv
def get_usernames_country_api():
    country_df = pd.read_csv(country_filepath)
    country_df

    for index, row in country_df.iterrows():        
        try:          
            country_name = row["Country"]
            country_code = row["Code"]            
            output_file_country = f"output_data/players_{country_code}.csv"

            get_usernames_country(country_code,output_file_country)
            # time.sleep(1)  

        except: 
            print("unepxected error/skipping country")

```{
  "avatar": "https://images.chesscomfiles.com/uploads/v1/user/14775486.9072453d.200x200o.50bb2d10a7d5.jpeg",
  "player_id": 14775486,
  "@id": "https://api.chess.com/pub/player/140mariam",
  "url": "https://www.chess.com/member/140mariam",
  "name": "Mariam Avetisyan",
  "username": "140mariam",
  "title": "WFM",
  "followers": 56,
  "country": "https://api.chess.com/pub/country/AM",
  "last_online": 1609780423,
  "joined": 1385829847,
  "status": "premium",
  "is_streamer": false
}
```

In [7]:
# check if output_data_file exists already, if it doesn't exist, do api call to get username data
#if (path.exists(output_data_players)):
    # ****************
    # pull prevously generated and saved output data from csv
    #chessplayer_df = pd.read_csv(output_data_players)
#else:    
    # run api calls for each chess.com username
    # this step is skipped if the output_data_file already exists
    # !!!!! DELETE CSV OUTPUT FILE to generate new playerdata    
    #playerdata = get_playerdata_api(player_usernames)
    # gets list of titled players
    #playerslist_data = get_titledplayers_api(titled_players_type)
    
    
    

In [8]:
## Get sample of users from csv file of one country (CANADA)
def get_sample_player_details(country):
    # country = "CA"
    players_country_file = f"output_data/players_{country}.csv"

    chessplayers_country_df = pd.read_csv(players_country_file)
    
    # get username count from country df
    chessplayers_count = len(chessplayers_country_df)    
    
    # if population is less than 5000 use population, if >= 5000 use 5000 as sample size
    if (chessplayers_count < 5000):        
        sample_count = chessplayers_count    
    if (chessplayers_count >= 5000):
        sample_count = 5000
        
    print(f"{sample_count} is the sample size of {country} population ({chessplayers_count})\n")
    data = np.random.randint(0, chessplayers_count, size=sample_count)

    sample_country_usernames = []
    for index in data:    
        username = chessplayers_country_df.iloc[index]["username"]    
        sample_country_usernames.append(username)


    playerdata = get_playerdata_api(sample_country_usernames)
    
    return playerdata


In [9]:
# CANADA - get player details from CANADA usernames
playerdata_CA = get_sample_player_details("CA")
player_CA_df = get_player_dataframe(playerdata_CA)
output_data_players_CA = 'output_data/player_details_CA.csv'
player_CA_df.to_csv(output_data_players_CA, index=True,index_label="player_index") 

In [13]:
# RUSSIA - get player details from RUSSIA usernames
playerdata_RU = get_sample_player_details("RU")
player_RU_df = get_player_dataframe(playerdata_RU)
output_data_players_RU = 'output_data/player_details_RU.csv'
player_RU_df.to_csv(output_data_players_RU, index=True,index_label="player_index") 

5000 is the sample size of RU population (103656)

Index 0 fetching player data for player ivanpetr
Index 1 fetching player data for player hukycenok
Index 2 fetching player data for player rst0000
Index 3 fetching player data for player art_em_up
Index 4 fetching player data for player boltina
Index 5 fetching player data for player 89blues
Index 6 fetching player data for player afanasij1914
Index 7 fetching player data for player jessie108
Index 8 fetching player data for player asveider
Index 9 fetching player data for player zolrus16
Index 10 fetching player data for player shved1974
Index 11 fetching player data for player pobeditel_08
Index 12 fetching player data for player mihburkov
Index 13 fetching player data for player brov79
Index 14 fetching player data for player kachviktor
Index 15 fetching player data for player peaceful0warrior
Index 16 fetching player data for player juniorzebra
Index 17 fetching player data for player vkurchatkin
Index 18 fetching player data for p

Index 157 fetching player data for player protopop77
Index 158 fetching player data for player gr4xx
Index 159 fetching player data for player sash0milash
Index 160 fetching player data for player solari17
Index 161 fetching player data for player muromec13
Index 162 fetching player data for player olegb83
Index 163 fetching player data for player ametisty
Index 164 fetching player data for player mizzzzzzgun
Index 165 fetching player data for player komdizzle
Index 166 fetching player data for player deadly_donut
Index 167 fetching player data for player ratnikxyz
Index 168 fetching player data for player avg_protection
Index 169 fetching player data for player urigus
Index 170 fetching player data for player chessthenbless
Index 171 fetching player data for player laughingelk
Index 172 fetching player data for player andrewsharaev
Index 173 fetching player data for player yerand7
Index 174 fetching player data for player amyforeal2
Index 175 fetching player data for player andrey_chf

Index 313 fetching player data for player smile5598
Index 314 fetching player data for player xzadjen
Index 315 fetching player data for player niinada
Index 316 fetching player data for player venia74
Index 317 fetching player data for player eyerabbit
Index 318 fetching player data for player elis0086
Index 319 fetching player data for player serg400
Index 320 fetching player data for player alexandro1975
Index 321 fetching player data for player zanwtp
Index 322 fetching player data for player monahnestor
Index 323 fetching player data for player dabasov
Index 324 fetching player data for player vanyamasterninja
Index 325 fetching player data for player marivylegzhanina
Index 326 fetching player data for player sovloki
Index 327 fetching player data for player kirmodin
Index 328 fetching player data for player desperadodanil
Index 329 fetching player data for player alekseymar
Index 330 fetching player data for player sergeykoshev
Index 331 fetching player data for player ckyhc123
I

Index 470 fetching player data for player brutal_kolobok
Index 471 fetching player data for player monica893
Index 472 fetching player data for player valeeriyaa
Index 473 fetching player data for player egorka635
Index 474 fetching player data for player psinneraf
Index 475 fetching player data for player dm0705
Index 476 fetching player data for player zainulla
Index 477 fetching player data for player rudenokslava
Index 478 fetching player data for player k_mike
Index 479 fetching player data for player shingenn
Index 480 fetching player data for player midnoon
Index 481 fetching player data for player mirkin4
Index 482 fetching player data for player sharkness007
Index 483 fetching player data for player gary0081
Index 484 fetching player data for player alaltru
Index 485 fetching player data for player greshnov_ivan
Index 486 fetching player data for player efremova_vii
Index 487 fetching player data for player taykov
Index 488 fetching player data for player mctatarochka
Index 48

Index 626 fetching player data for player adeliya1822
Index 627 fetching player data for player lak1n
Index 628 fetching player data for player civilcase
Index 629 fetching player data for player vovamoroz
Index 630 fetching player data for player lapin_sergey
Index 631 fetching player data for player s_konem_naa8
Index 632 fetching player data for player stepanpotey-ipa
Index 633 fetching player data for player kirrees
Index 634 fetching player data for player arinakov
Index 635 fetching player data for player rei3772
Index 636 fetching player data for player georgyefanov
Index 637 fetching player data for player geymonster
Index 638 fetching player data for player coffee-zombie
Index 639 fetching player data for player dimadyrygin
Index 640 fetching player data for player valchess13
Index 641 fetching player data for player daniil-n
Index 642 fetching player data for player eclatmk
Index 643 fetching player data for player grossmeister2019
Index 644 fetching player data for player al

...

Index 4469 fetching player data for player meta_cube
Index 4470 fetching player data for player zzzzzzelten
Index 4471 fetching player data for player iigoruha
Index 4472 fetching player data for player chessclub_64
Index 4473 fetching player data for player masimilian
Index 4474 fetching player data for player dmitriyvdk
Index 4475 fetching player data for player vladimirsurov
Index 4476 fetching player data for player fosfor195
Index 4477 fetching player data for player sputer22
Index 4478 fetching player data for player 1802a
Index 4479 fetching player data for player spiffymarble
Index 4480 fetching player data for player yafizzzov
Index 4481 fetching player data for player mymateturn
Index 4482 fetching player data for player mikhailnaumov27
Index 4483 fetching player data for player alexxxan2610
Index 4484 fetching player data for player quiet_hour
Index 4485 fetching player data for player shomilinho
Index 4486 fetching player data for player greenliner
Index 4487 fetching playe

Index 4621 fetching player data for player trolliwally
Index 4622 fetching player data for player romanvlasoff
Index 4623 fetching player data for player amarei77
Index 4624 fetching player data for player sem1465
Index 4625 fetching player data for player roza0770
Index 4626 fetching player data for player kerij14
Index 4627 fetching player data for player vdergachoff
Index 4628 fetching player data for player haimkat
Index 4629 fetching player data for player mikhailmovchan
Index 4630 fetching player data for player dunga63
Index 4631 fetching player data for player chessmouse1
Index 4632 fetching player data for player winchester1388
Index 4633 fetching player data for player kirka377
Index 4634 fetching player data for player savvaloginov
Index 4635 fetching player data for player murphqw
Index 4636 fetching player data for player biatlonist10
Index 4637 fetching player data for player liquipups
Index 4638 fetching player data for player diannadilaila
Index 4639 fetching player dat

Index 4775 fetching player data for player chattybagpipe
Index 4776 fetching player data for player toria111
Index 4777 fetching player data for player cheesmaster777777
Index 4778 fetching player data for player artik353
Index 4779 fetching player data for player originalzipper
Index 4780 fetching player data for player netnet1048
Index 4781 fetching player data for player raccoon6e
Index 4782 fetching player data for player alimkozhenov
Index 4783 fetching player data for player deni5018
Index 4784 fetching player data for player vyurok
Index 4785 fetching player data for player serg1977786
Index 4786 fetching player data for player 414ok
Index 4787 fetching player data for player nastia565
Index 4788 fetching player data for player ckyhc123
Index 4789 fetching player data for player tired_tiger
Index 4790 fetching player data for player vaili
Index 4791 fetching player data for player milena1415
Index 4792 fetching player data for player ramzessecond
Index 4793 fetching player data 

Index 4927 fetching player data for player debuer77
Index 4928 fetching player data for player aram_tsamtsam
Index 4929 fetching player data for player valerii1976
Index 4930 fetching player data for player feliksburtyuk
Index 4931 fetching player data for player susemka
Index 4932 fetching player data for player lissinichkin
Index 4933 fetching player data for player smirnovs3000
Index 4934 fetching player data for player zverev49
Index 4935 fetching player data for player mmm235
Index 4936 fetching player data for player escobar38
Index 4937 fetching player data for player victor18041980
Index 4938 fetching player data for player valllery
Index 4939 fetching player data for player thunderdamn76
Index 4940 fetching player data for player kirillmed1985
Index 4941 fetching player data for player aliagalimova
Index 4942 fetching player data for player qw333
Index 4943 fetching player data for player i580
Index 4944 fetching player data for player maxwill90
Index 4945 fetching player data

Index 3421 processing username my126
Index 3422 processing username dug_o
Index 3423 processing username sergei-po-nastroeniu
Index 3424 processing username liarm
Index 3425 processing username 5bi4
Index 3426 processing username sablin_alexey
Index 3427 processing username yfa102102
Index 3428 processing username lada2703
Index 3429 processing username mczenit
Index 3430 processing username peterprigor
Index 3431 processing username ilyas_nix
Index 3432 processing username coolsurgion
Index 3433 processing username begemot8
Index 3434 processing username therealmast
Index 3435 processing username andrewsudarikov
Index 3436 processing username sasharulit1
Index 3437 processing username alinatret
Index 3438 processing username katsu_nyan
Index 3439 processing username dimkov2
Index 3440 processing username infud
Index 3441 processing username craxulty
Index 3442 processing username vegetut
Index 3443 processing username bonitoy
Index 3444 processing username vasiliyer
Index 3445 process

In [14]:
# MEXICO - get player details from MEXICO usernames
playerdata_MX = get_sample_player_details("MX")
player_MX_df = get_player_dataframe(playerdata_MX)
output_data_players_MX = 'output_data/player_details_MX.csv'
player_MX_df.to_csv(output_data_players_MX, index=True,index_label="player_index") 

5000 is the sample size of MX population (65853)

Index 0 fetching player data for player josuegch
Index 1 fetching player data for player pacopac1
Index 2 fetching player data for player kikequesos
Index 3 fetching player data for player rick_03
Index 4 fetching player data for player raulhernandezgarcia
Index 5 fetching player data for player penetreitor235
Index 6 fetching player data for player chnoriega
Index 7 fetching player data for player danielcdl
Index 8 fetching player data for player alemanias
Index 9 fetching player data for player j_rendon98
Index 10 fetching player data for player pablo1188
Index 11 fetching player data for player cvazquez48
Index 12 fetching player data for player chubo1376
Index 13 fetching player data for player deambulnte
Index 14 fetching player data for player llockdownl
Index 15 fetching player data for player reginamestre
Index 16 fetching player data for player warriorgod333
Index 17 fetching player data for player angelbaeza777
Index 18 fetchi

Index 156 fetching player data for player arlong-d
Index 157 fetching player data for player cocol81
Index 158 fetching player data for player yaz0808
Index 159 fetching player data for player salcedo13
Index 160 fetching player data for player hktr90
Index 161 fetching player data for player karis004
Index 162 fetching player data for player aramhub
Index 163 fetching player data for player babyjey69
Index 164 fetching player data for player ehrodgz
Index 165 fetching player data for player imiazu
Index 166 fetching player data for player archiame
Index 167 fetching player data for player cheeringpasture
Index 168 fetching player data for player rolos84
Index 169 fetching player data for player knigh604
Index 170 fetching player data for player rdavilac
Index 171 fetching player data for player duende7
Index 172 fetching player data for player romanzm
Index 173 fetching player data for player frodguz
Index 174 fetching player data for player cliffor1
Index 175 fetching player data for

Index 312 fetching player data for player monkeykingelian
Index 313 fetching player data for player carlos_palacios
Index 314 fetching player data for player jonathanantonio
Index 315 fetching player data for player servingardea
Index 316 fetching player data for player bruce5011
Index 317 fetching player data for player jefejime
Index 318 fetching player data for player nani2737
Index 319 fetching player data for player scroud_83
Index 320 fetching player data for player alucard1208z
Index 321 fetching player data for player peonch3ss
Index 322 fetching player data for player javiervallejo
Index 323 fetching player data for player davidsea27089
Index 324 fetching player data for player yayanman
Index 325 fetching player data for player itzy310
Index 326 fetching player data for player villegas72
Index 327 fetching player data for player fernandosaid7
Index 328 fetching player data for player argenta2k
Index 329 fetching player data for player horaceleon
Index 330 fetching player data 

Index 468 fetching player data for player medrano_2099
Index 469 fetching player data for player el_centinela
Index 470 fetching player data for player yorick_el_bufon
Index 471 fetching player data for player jonathancn16
Index 472 fetching player data for player julian_2
Index 473 fetching player data for player ortizkey
Index 474 fetching player data for player landarck
Index 475 fetching player data for player alucard245
Index 476 fetching player data for player katiarlen
Index 477 fetching player data for player veeadams12
Index 478 fetching player data for player angelgs94
Index 479 fetching player data for player jsehxl
Index 480 fetching player data for player spartanote
Index 481 fetching player data for player alansync
Index 482 fetching player data for player glm2020
Index 483 fetching player data for player zottacko
Index 484 fetching player data for player mauricio202
Index 485 fetching player data for player jara2811
Index 486 fetching player data for player nobodylikesri

Index 624 fetching player data for player tiradoo
Index 625 fetching player data for player ricardomich
Index 626 fetching player data for player franz0075
Index 627 fetching player data for player jemsego
Index 628 fetching player data for player beta7
Index 629 fetching player data for player kat1612
Index 630 fetching player data for player muchomejorquerana
Index 631 fetching player data for player ponchotorres
Index 632 fetching player data for player kylerapps
Index 633 fetching player data for player isaama51
Index 634 fetching player data for player telvannimage
Index 635 fetching player data for player nahumrebolledo
Index 636 fetching player data for player ramsesponce
Index 637 fetching player data for player elingeniebrio
Index 638 fetching player data for player daxmer
Index 639 fetching player data for player ariel5607
Index 640 fetching player data for player enrique1895
Index 641 fetching player data for player croacktable
Index 642 fetching player data for player cooki

...

Index 4456 fetching player data for player cristocalzada
Index 4457 fetching player data for player travis2359
Index 4458 fetching player data for player adolfo_night
Index 4459 fetching player data for player franciscobernal7
Index 4460 fetching player data for player fabricio10
Index 4461 fetching player data for player antua2
Index 4462 fetching player data for player juggernaut61
Index 4463 fetching player data for player ramiguel8
Index 4464 fetching player data for player morrinni
Index 4465 fetching player data for player rgonzalezvega1
Index 4466 fetching player data for player 1409mich
Index 4467 fetching player data for player chequeee
Index 4468 fetching player data for player safari66
Index 4469 fetching player data for player masantor
Index 4470 fetching player data for player capi1003
Index 4471 fetching player data for player jmreyesrugerio
Index 4472 fetching player data for player armandopaquini
Index 4473 fetching player data for player jabond27
Index 4474 fetching pl

Index 4610 fetching player data for player lunarooyoss
Index 4611 fetching player data for player johanbueno
Index 4612 fetching player data for player linko17
Index 4613 fetching player data for player alex6995
Index 4614 fetching player data for player diegoramrezd
Index 4615 fetching player data for player ivan3075
Index 4616 fetching player data for player cesmat367
Index 4617 fetching player data for player guillermo1504
Index 4618 fetching player data for player fabianmontano
Index 4619 fetching player data for player isaacmejia23
Index 4620 fetching player data for player kraknik
Index 4621 fetching player data for player rafarlezama
Index 4622 fetching player data for player abel_1618
Index 4623 fetching player data for player rafacs22
Index 4624 fetching player data for player wizhard2705
Index 4625 fetching player data for player elpuma1981
Index 4626 fetching player data for player roob16
Index 4627 fetching player data for player diego3174
Index 4628 fetching player data fo

Index 4763 fetching player data for player efrainameca
Index 4764 fetching player data for player alberthoffman11
Index 4765 fetching player data for player rolex2409
Index 4766 fetching player data for player luisruvalcabachavez
Index 4767 fetching player data for player emiliano7752
Index 4768 fetching player data for player gengis66
Index 4769 fetching player data for player superchessy2019
Index 4770 fetching player data for player themexicantarzan
Index 4771 fetching player data for player horaciooropeza
Index 4772 fetching player data for player mollejascastro
Index 4773 fetching player data for player oscarert02
Index 4774 fetching player data for player calderongx
Index 4775 fetching player data for player jalomojalomo97
Index 4776 fetching player data for player steeler1961
Index 4777 fetching player data for player ebaez12
Index 4778 fetching player data for player melquizedecm
Index 4779 fetching player data for player braku619
Index 4780 fetching player data for player osca

Index 4914 fetching player data for player eduardodom
Index 4915 fetching player data for player yvavc
Index 4916 fetching player data for player urielruano
Index 4917 fetching player data for player lunabonita
Index 4918 fetching player data for player vtec
Index 4919 fetching player data for player damislove
Index 4920 fetching player data for player sirpuckling
Index 4921 fetching player data for player everastorga
Index 4922 fetching player data for player satangalvan
Index 4923 fetching player data for player 3dher18
Index 4924 fetching player data for player alexiscasasquintanar
Index 4925 fetching player data for player feral662
Index 4926 fetching player data for player alfonsolopez2
Index 4927 fetching player data for player luis230602
Index 4928 fetching player data for player zars0
Index 4929 fetching player data for player cesa0279
Index 4930 fetching player data for player kenblox
Index 4931 fetching player data for player cmadachess
Index 4932 fetching player data for pla

Index 3355 processing username mesias2353
Index 3356 processing username yourdaddy42
Index 3357 processing username fanny7067
Index 3358 processing username sag2173
Index 3359 processing username hambertoy
Index 3360 processing username xavaaudelo
Index 3361 processing username xabovar
Index 3362 processing username milan2410
Index 3363 processing username diiannaxd
Index 3364 processing username s3b4st
Index 3365 processing username sani157
Index 3366 processing username enriqueadabache
Index 3367 processing username jorgeelcapi3240
Index 3368 processing username josuejs
Index 3369 processing username kaijuking07
Index 3370 processing username omarpaniagua
Index 3371 processing username loky78
Index 3372 processing username victor_de_leon
Index 3373 processing username angelruizm
Index 3374 processing username jesusmoya921
Index 3375 processing username eayonm
Index 3376 processing username agedcrane6
Index 3377 processing username ivandraggo23
Index 3378 processing username lrgckp
In